In [102]:
from XCD.IRT.MXIRT import IRT

In [103]:
irt = IRT.train(
    "../../../data/LSAT/train.csv",
    "../../../data/LSAT/valid.csv",
    model_dir="../../../data/LSAT/model",
    hyper_params_update=dict(user_num=1000, item_num=5),
    # init_params_update={"initial_user_item": False},
    batch_size=64,
)

MXIRT, INFO Params
logger: <Logger MXIRT (INFO)>
model_name: MXIRT
root: E:\Program\XCD
dataset: 
timestamp: 20201123222236
workspace: 
root_data_dir: E:\Program\XCD\data
data_dir: E:\Program\XCD\data\data
root_model_dir: E:\Program\XCD\data\model\MXIRT
model_dir: ../../../data/LSAT/model
cfg_path: ..\..\..\data\LSAT\model\configuration.json
begin_epoch: 0
end_epoch: 10
batch_size: 64
save_epoch: 1
optimizer: adam
optimizer_params: {'learning_rate': 0.01, 'clip_gradient': 1}
lr_params: {'update_params': {}}
train_select: None
save_select: None
ctx: cpu(0)
toolbox_params: {}
hyper_params: {'user_num': 1000, 'item_num': 5, 'theta_range': [-4, 4], 'a_range': [0.25, 2], 'b_range': [-2, 2], 'c_range': [0, 0.4]}
init_params: {}
loss_params: {}
caption: 
hyper_params_update: {'user_num': 1000, 'item_num': 5}
validation_result_file: ..\..\..\data\LSAT\model\result.json
MXIRT, INFO parameters will be saved to ../../../data/LSAT/model
MXIRT, INFO generating symbol
MXIRT, INFO loading data
MXIRT,

Epoch| Total-E          Batch     Total-B       Loss-LogisticLoss             Progress           
    0|       9             53          53                0.448159    [00:00<00:00, 175.39it/s]     
Epoch [0]	Train Time-0.303s	Loss - LogisticLoss: 0.448159
           precision  recall        f1  support
0.0         0.000000     0.0  0.000000       58
1.0         0.845333     1.0  0.916185      317
macro_avg   0.422667     0.5  0.458092      375
accuracy: 0.845333	macro_auc: 0.660122	macro_aupoc: 0.913588
Epoch| Total-E          Batch     Total-B       Loss-LogisticLoss             Progress           
    1|       9             53          53                0.440803    [00:00<00:00, 176.55it/s]   
Epoch [1]	Train Time-0.301s	Loss - LogisticLoss: 0.440803
           precision    recall        f1  support
0.0         0.200000  0.017241  0.031746       58
1.0         0.845946  0.987382  0.911208      317
macro_avg   0.522973  0.502312  0.471477      375
accuracy: 0.837333	macro_auc: 0.67953

In [104]:
# use longling to find best epoch
!longling amax macro_auc macro_aupoc --src ../../../data/LSAT/model/result.json --with_keys "Epoch;macro_auc;macro_aupoc"

             Epoch  macro_auc  macro_aupoc
macro_auc        1   0.679539      0.91932
macro_aupoc      1   0.679539      0.91932


In [105]:
# switch to the best epoch
irt.reload(1)
# get user_params and item_params
user_df, item_df = irt()
item_df.describe()
user_df.describe()

MXIRT, INFO load params from existing model file ../../../data/LSAT/model\MXIRT-0001.parmas


learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


,user_id,theta
count,1000.000000,1000.000000
mean,499.500000,0.847231
std,288.819436,0.362693
min,0.000000,-0.298079
25%,249.750000,0.631132
50%,499.500000,0.877015
75%,749.250000,1.169003
max,999.000000,1.301321


In [106]:
user_df.head()

,user_id,theta
0,0,-0.163669
1,1,-0.298079
2,2,-0.203258
3,3,0.285473
4,4,-0.014447


In [107]:
item_df.head()

,item_id,a,b,c
0,0,1.718022,-0.760082,0.400000
1,1,1.709172,-0.323031,0.400000
2,2,1.784342,0.203941,0.112781
3,3,1.712841,-0.566244,0.400000
4,4,1.668242,-0.634353,0.400000


In [108]:
import pandas as pd
test_df = pd.read_csv("../../../data/LSAT/test.csv")
test_df.head()

,user_id,item_id,score
0,478,3,1
1,740,3,1
2,467,0,1
3,870,2,1
4,918,4,1


In [109]:
from XCD.IRT.metrics import eval_irt

eval_irt(test_df, user_df, item_df)


           precision    recall        f1  support
0           0.333333  0.010830  0.020979      277
1           0.779210  0.993834  0.873532      973
macro_avg   0.556272  0.502332  0.447256     1250
accuracy: 0.776000	macro_auc: 0.673990	macro_aupoc: 0.868745
